Касьянов Илья

Задание D.  (8-12 баллов)

Составить программу, выполняющую извлечение из текста на русском языке устойчивых 	словосочетаний. Для этого:

* Выбрать довольно объемный текст на русском языке и провести его лемматизацию;
* Определить частотное слово данного текста (существительное, прилагат., глагол);
* Подсчитать частоту встречаемости биграмм с этим словом; 
* Извлечь список из 15-20 наиболее устойчивых биграмм по мере Дайса и по мере MI;
* Сравнить и прокомментировать извлеченные списки, упорядоченные по значению меры.

Отчет: Описание составленной программы (ее текст с комментариями);  указание вычисленного частотного слова; извлеченные списки биграмм со	значениями мер; выводы по проведенному сравнению.

In [15]:
import pymorphy2, os, re
# будем использовать pymorphy2 для последующей лемматизаци слов
morph = pymorphy2.MorphAnalyzer()

# https://www.e-reading.club/book.php?book=147848
# book_name = 'Адамс_-_Путеводитель_“Автостопом_по_Галактике”.txt'
book_name = 'Томас Мор. Утопия - royallib.ru.txt'
book_text = open(os.path.join(os.getcwd(), book_name), 'r').read()

# функция лемматизации для слова проводит его морфологический анализ
# и выдает пару, состоящую из
# * предполагаемой нормальной формы слова;
# * соответствующей ему части речи;
lemma = lambda word: (morph.parse(word)[0].normal_form, morph.parse(word)[0].tag.POS)
# (мы будем выбирать наиболее вероятный морфологический разбор,
# поскольку мы собираемся работать не со словоформами, но со словами, и определять их частотность:
# это удобно делать, поставив каждой словоформе независимо от контекста её нормальную форму;
# для задания нам потребуется также хранить часть речи
# (еще это поможет разделять слова, чьи наиболее вероятные нормальные формы совпадают:
# рабочий (прил.) - рабочий (сущ.), например)

# проведем "наивное" разбиение текста на слова при помощи регулярных выражений,
book_words = re.findall(r'\w+', book_text)
# лемматизируем при помощи функции lemma
# и заполним словарь абсолютных частот
# {лексема, выраженная нормальной формой + частью речи : абсолютная частота}
freq_dict = {}
for word in book_words:
    l = lemma(word)
    freq_dict[l] = freq_dict[l] + 1 if l in freq_dict.keys() else 1

In [16]:
# из полученного словаря сделаем упорядоченные списки для каждой части речи
# по убыванию абсолютных частот {часть речи : {норм. форма : абсолютная частота}}
freq_lists = {POS : sorted([(p[0][0], p[1]) for p in freq_dict.items() if p[0][1] == POS],
                           key = lambda x: x[1],
                           reverse = True)
              for POS in set(key[1] for key in freq_dict.keys())}
freq_lists["NOUN"][:10], freq_lists["VERB"][:10], freq_lists["ADJF"][:10]

([('утопиец', 187),
  ('человек', 182),
  ('год', 137),
  ('народ', 115),
  ('имя', 107),
  ('жизнь', 104),
  ('мор', 102),
  ('друг', 100),
  ('город', 94),
  ('случай', 92)],
 [('быть', 327),
  ('мочь', 164),
  ('нибыть', 71),
  ('иметь', 58),
  ('считать', 42),
  ('иметься', 41),
  ('являться', 36),
  ('стать', 35),
  ('признавать', 31),
  ('знать', 24)],
 [('тот', 379),
  ('весь', 344),
  ('который', 326),
  ('свой', 285),
  ('этот', 244),
  ('один', 234),
  ('другой', 215),
  ('такой', 207),
  ('какой', 183),
  ('самый', 112)])

In [23]:
# найденное частотное существительное
# freq_word = (freq_lists['NOUN'][0][0], 'NOUN')
freq_word = ('человек', 'NOUN')
print(freq_word[0], freq_dict[freq_word])

человек 182


In [24]:
# составим частотный словарь для упорядоченных биграмм с заданным частотным словом freq_word:
# ключами будут упорядоченные нормальные формы словоформ биграмм с их частями речи, 
# значениями - массивы употребления данной биграммы в оригинальном тексте (т.е. с изначальными словоформами),
# длины этих массивов и будут являться абсолютными частотами употребления биграмм
def bigramms(freq_word):
    bi_freq_dict = {}
    for index, word in enumerate(book_words):
        lemma_w = lemma(word) 
        if lemma_w == freq_word:
            bigramms = []
            if index > 0:
                bigramms.append(((lemma(book_words[index - 1]), lemma_w),
                                 book_words[index - 1] + " " + word))
            # + условие на то, чтобы не считать биграммы вида (lemma_w, lemma_w) два раза
            if index < len(book_words) - 1 and lemma(book_words[index + 1]) != lemma_w:
                bigramms.append(((lemma_w, lemma(book_words[index + 1])),
                                  word + " " + book_words[index + 1]))
            for b in bigramms:
                if b[0] not in bi_freq_dict.keys():
                    bi_freq_dict[b[0]] = [b[1]]
                else:
                    bi_freq_dict[b[0]].append(b[1]) 
            del bigramms
    return bi_freq_dict
bi_freq_dict = bigramms(freq_word)

In [25]:
# посмотрим на упорядоченный по частоте список биграмм с заданным частотным словом freq_word:
# для наглядности заменим биграммы, составленные из лемм, на примеры их употребления value[0]
# (совпадение примеров употребления гарантирует совпадение ключей-"нормальных форм")
bi_freq_list = {value[0]:len(value) for key, value in bi_freq_dict.items()}
bi_freq_list = list(sorted(bi_freq_list.items(), key = lambda x : x[1], reverse = True))
bi_freq_list[:20]

[('человека который', 12),
 ('такого человека', 9),
 ('людьми не', 8),
 ('человек и', 8),
 ('эти люди', 7),
 ('человек а', 7),
 ('людям в', 7),
 ('одних людей', 6),
 ('хороших людей', 6),
 ('у человека', 4),
 ('людьми они', 4),
 ('даже людей', 4),
 ('самые люди', 4),
 ('людей по', 4),
 ('этого человека', 4),
 ('человека с', 4),
 ('людей этого', 4),
 ('для людей', 3),
 ('людей на', 3),
 ('людей он', 3)]

In [37]:
import math
# зададим функции мер устойчивости для биграмм,
# построим словари, сопоставляющие биграммам:
# * их абсолютные частоты;
# * абсолютные частоты слов, которые в них входят
# * их меры Дайса и MI;
# и из полученных словарей сделаем упорядоченные списки по убыванию мер
# (для наглядности мы в качестве биграммы рассматриваем один из примеров её употребления value[0])
def bigramms_measures(bi_freq_dict):    
    Dice = lambda bigramm: 2 * len(bi_freq_dict[bigramm]) / (freq_dict[bigramm[0]] + freq_dict[bigramm[1]])
    MI = lambda bigramm: math.log((len(book_words) * len(bi_freq_dict[bigramm])) /
                                  (2 * freq_dict[bigramm[0]] * freq_dict[bigramm[1]]))
    dice_mi_dict = {value[0]:{'freqs': (len(value), (freq_dict[key[0]], freq_dict[key[1]])),
                              'Dice' : Dice(key), 'MI' : MI(key)} for key, value in bi_freq_dict.items()}
    dice_list = list(sorted(dice_mi_dict.items(), key = lambda x : x[1]['Dice'], reverse = True))
    mi_list = list(sorted(dice_mi_dict.items(), key = lambda x : x[1]['MI'], reverse = True))
    return dice_list, mi_list
dice_list, mi_list = bigramms_measures(bi_freq_dict)

In [42]:
bi_freq_dict[lemma('хороших'), lemma('людей')]

['хороших людей',
 'хороших людей',
 'лучшими людьми',
 'хороших людей',
 'хороших человек',
 'хороших людей']

Итак, извлечем список из 20 наиболее устойчивых биграмм с частотным словом по мере Дайса и по мере MI.
Пежде, чем приступать к сравнению полученных списков, вспомним, что показывают эти меры устойчивости.

Обозначим за $F_{bi}$ функцию частотности биграмм, то есть частоту совместной встречаемости двух слов, за $F$ - функцию абсолютной частоты употребления (эти функции соответствуют нашим словарям $\textit{bi_freq_dict}$ и $\textit{freq_dict}$ соответственно):
* $\textbf{Dice(a_1, a_2)} = \dfrac{2 \cdot F_{bi}(a_1, a_2)}{F(a_1) + F(a_2)} \in [0, 1]$

мера $\text{Dice}$ принимает значение $0$, если слова $a_1$ и $a_2$ друг с другом не встречаются вообще, и $1$, если слова $a_1$ и $a_2$ встречаются исключительно друг с другом (так как в таком случае $F_{bi}(a_1, a_2) = F(a_1) = F(a_2)$);  
эта мера показывает вероятность нахождения двух слов вместе, так как $2 \cdot F_{bi}(a_1, a_2)$ - количество таких употреблений слова $a_1$ или $a_2$, что слово $a_2$ или $a_1$ соответственно находится рядом, а $F(a_1) + F(a_2)$ - общее количество употреблений обоих слов.
* $\textbf{MI(a_1, a_2)} = \log_2{\dfrac{N \cdot F_{bi}(a_1, a_2)}{F(a_1) \cdot F(a_2)}}$, где $N$ - число слов $(N = len(\textit{book_words)} = sum(\textit{freq_dict.values()}))$

мера $\text{MI}$ принимает значение $-\infty$, если слова $a_1$ и $a_2$ друг с другом не встречаются вообще, и $\log_2{\dfrac{N}{F(a_2)}}$, если слово $a_1$ встречается исключительно с $a_2$ (при $F(a_1) \leq F(a_2)$ это максимальное значение $\textbf{MI}$ при постоянных $F(a_1)$, $F(a_2)$, так как $F_{bi}(a_1, a_2) \leq \min(F(a_1), F(a_2)) \Longrightarrow \text{MI(a_1, a_2)} <= \log_2{\frac{N}{\max(F(a_1), F(a_2))}}$);  
эта мера позволяет оценить влияние появления одного слова на появление другого, так как $\text{MI(a_1, a_2)} = \log_2{\frac{N}{F_{bi}(a_1, a_2)}} + \log_2{\frac{F_{bi}(a_1, a_2)}{F(a_1)}} + \log_2{\frac{F_{bi}(a_1, a_2)}{F(a_2)}}$, то есть биграмма тем устойчивее, чем больше $\frac{F_{bi}(a_1, a_2)}{F(a_i)}$ - вероятность того, что $a_i$ состоит в биграмме $(a_1, a_2)$.

In [27]:
# для начала посмотрим на биграммы, отсортированные по мере MI
mi_list[:20]

[('люди рвутся',
  {'Dice': 0.01092896174863388,
   'MI': 4.77546921145867,
   'freqs': (1, (182, 1))}),
 ('людям держащимся',
  {'Dice': 0.01092896174863388,
   'MI': 4.77546921145867,
   'freqs': (1, (182, 1))}),
 ('аббаты люди',
  {'Dice': 0.01092896174863388,
   'MI': 4.77546921145867,
   'freqs': (1, (1, 182))}),
 ('отвращают людей',
  {'Dice': 0.01092896174863388,
   'MI': 4.77546921145867,
   'freqs': (1, (1, 182))}),
 ('ожесточает человека',
  {'Dice': 0.01092896174863388,
   'MI': 4.77546921145867,
   'freqs': (1, (1, 182))}),
 ('долой людей',
  {'Dice': 0.01092896174863388,
   'MI': 4.77546921145867,
   'freqs': (1, (1, 182))}),
 ('человек честный',
  {'Dice': 0.01092896174863388,
   'MI': 4.77546921145867,
   'freqs': (1, (182, 1))}),
 ('простодушным людям',
  {'Dice': 0.01092896174863388,
   'MI': 4.77546921145867,
   'freqs': (1, (1, 182))}),
 ('потоптали люди',
  {'Dice': 0.01092896174863388,
   'MI': 4.77546921145867,
   'freqs': (1, (1, 182))}),
 ('связует человека',
  

Зафиксируем одно из слов.  
Так как для мер не играет роли порядок слов в биграмме, будем считать, что зафиксированно слово $a_1$. Тогда:  
$\text{MI(a_1, a_2)} = \log_2{\frac{N}{F(a_1)}} + \log_2{\frac{F_{bi}(a_1, a_2)}{F(a_2)}}$, т.е. при сравнении по этой мере различных биграмм с фиксированным словом мы сравниваем вероятность того, что второе слово состоит в биграмме с ним, - чем больше эта вероятность, тем больше соответствующая мера.

Посольку наибольшее значение этой вероятности равно один, то наиболее устойчивыми словосочетаниями по мере $\text{MI}$ будут являться биграммы с уникальным для текста $a_2$ (т.е. $F(a_2) = F_{bi}(a_1, a_2) = 1$), - именно этот эффект и демонстрирует выделенный нами список.

In [34]:
# теперь посмотрим на биграммы, отсортированные по мере Dice
dice_list[:20]

[('хороших людей',
  {'Dice': 0.05309734513274336,
   'MI': 2.7830390467684643,
   'freqs': (6, (44, 182))}),
 ('человека который',
  {'Dice': 0.047244094488188976,
   'MI': 1.4734784798799634,
   'freqs': (12, (182, 326))}),
 ('такого человека',
  {'Dice': 0.04627249357326478,
   'MI': 1.6399749955295209,
   'freqs': (9, (207, 182))}),
 ('эти люди',
  {'Dice': 0.03286384976525822,
   'MI': 1.224211135220782,
   'freqs': (7, (244, 182))}),
 ('богатых людей',
  {'Dice': 0.030927835051546393,
   'MI': 3.38917485033878,
   'freqs': (3, (12, 182))}),
 ('даже людей',
  {'Dice': 0.030418250950570342,
   'MI': 1.7673144179061224,
   'freqs': (4, (81, 182))}),
 ('человека вообще',
  {'Dice': 0.028985507246376812,
   'MI': 2.6552056752585793,
   'freqs': (3, (182, 25))}),
 ('одних людей',
  {'Dice': 0.028846153846153848,
   'MI': 1.1119075653290242,
   'freqs': (6, (234, 182))}),
 ('самые люди',
  {'Dice': 0.027210884353741496,
   'MI': 1.4432647012834667,
   'freqs': (4, (112, 182))}),
 ('чело

Обозначим за $F_{bi}(a_i, \cdot)$ количество всех биграмм со словом $a_i$. Это количество, согласно написанной нами программе, равняется либо $2 \cdot F(a_i)$, либо $2 \cdot F(a_i) + 1$, либо $2 \cdot F(a_i) + 2$ (последние два варианта связаны с употреблением этого слова в начале или конце текста). Пусть $$P_{a_i}(a_j) = \dfrac{F_{bi}(a_i, a_j)}{F_{bi}(a_i, \cdot)} = \dfrac{F_{bi}(a_i, a_j)}{2 \cdot F(a_i)}$$ - относительная частота биграммы $(a_i, a_j)$ в наборе биграмм слова $a_i$, которое не стоит в начале или в конце текста.

В отличие от меры $\text{MI}$, которая сравнивает $\frac{F_{bi}(a_1, a_2)}{F(a_2)} = 2 \cdot P_{a_2}(a_1)$, $\text{Dice}$ добавляет в это выражение абсолютную частоту зафиксированного слова и сравнивает уже выражения $\frac{2 \cdot F_{bi}(a_1, a_2)}{F(a_1) + F(a_2)} = \frac{1}{\frac{1}{P_{a_2}(a_1)} + \frac{1}{P_{a_1}(a_2)}}$. Следовательно, сравнение по мере $\text{Dice}$, в отличие от меры $MI$, означает зависимость не только от относительной частоты биграммы в наборе биграмм незафиксированного слова, но и от ее относительной частоты во всех биграммах фиксированного слова, - увеличение этих частот означает увеличение $\text{Dice}$. Нельзя говорить об обратной зависимости (увеличение $\text{Dice}$ не обязательно означает увеличение обеих относительных частот), но в любом случае, $\text{Dice}$ оценивает уже не просто влияние на появление второго слова, но частоту биграммы относительно частот всех биграмм с этими словами.

Условие на то, чтобы $a_i$ не стояло в начале и конце текста, достигается нами очень просто, - поскольку мы фиксируем наиболее частотное слово и анализируем все биграммы с ним, то мы можем в начало и конец текста добавить служебных слов так, чтобы слова из этих биграмм не стояли в начале или конце текста.

Наконец, дадим несколько комментариев относительно анализа книги Дугласа Адамса “Автостопом по Галактике”. В качестве частотного слова было выбрано существительное, и неудивительно, что им оказалось имя одного из главных героев (тем не менее, оно более конкретно, чем наиболее частотные глаголы и прилагательные). Наиболее устойчивые биграммы по мере MI являются уникальными по книге, т.е. второе слово используется только в одной биграмме с именем главного героя. Таких биграмм достаточно много, и не все они корректны - биграммы искались без учета знаков препинания, например, биграмма "вреда Артур" не является корректной - слово "вреда" по книге находится в предыдущем предложении.

Гораздо интереснее с точки зрения устойчивых выражений является сравнение по мере Дайса, которая учитывает не только относительную частоту биграммы в биграммах второго слова, но и первого: как можно видеть по полученному списку, мера MI для этих биграмм достаточно высока, а топ-5 устойчивых словосочетаний - это 'спросил Артур', 'Артур Дент', 'сказал Артур', 'Артура Дента', 'удивился Артур', - и это действительно похоже на устойчивые для данного текста словосочетания, глаголы, например, характеризуют "устойчивые" действия главного героя; 'Артур Дент' вовсе представляет собой имя и фамилию главного героя, что безуловно является устойчивым словосочетанием.

При этом в списке появляются служебные части речи; некоторые из них стоят вторыми и начинаются с заглавной буквы - несомненный признак того, что слова в этих биграммах были разделены точкой или символом конца предложения (судя по книге, в которой достаточно часты использования прямой речи, биграмма 'Артур Я' взялась явно оттуда, - почему бы не считать ее так же устойчивой).

In [33]:
# поиграемся немного с теми упорядоченными по мерам списками, которые мы получили
# выведем, например, такие биграммы, упорядоченные по мере MI, что второе слово встречается в тексте чаще одного раза
[b for b in mi_list if b[1]['freqs'][0] > 1][:20]

[('понятие человека',
  {'Dice': 0.021739130434782608,
   'MI': 4.77546921145867,
   'freqs': (2, (2, 182))}),
 ('темных людей',
  {'Dice': 0.021505376344086023,
   'MI': 4.082322030898725,
   'freqs': (2, (4, 182))}),
 ('людей вышедшем',
  {'Dice': 0.02127659574468085,
   'MI': 3.6768569227905608,
   'freqs': (2, (182, 6))}),
 ('подобно человеку',
  {'Dice': 0.021052631578947368,
   'MI': 3.38917485033878,
   'freqs': (2, (8, 182))}),
 ('богатых людей',
  {'Dice': 0.030927835051546393,
   'MI': 3.38917485033878,
   'freqs': (3, (12, 182))}),
 ('люди отличаются',
  {'Dice': 0.020942408376963352,
   'MI': 3.2713918146823966,
   'freqs': (2, (182, 9))}),
 ('знание людей',
  {'Dice': 0.02072538860103627,
   'MI': 3.070721119220245,
   'freqs': (2, (11, 182))}),
 ('свободным человеком',
  {'Dice': 0.020618556701030927,
   'MI': 2.9837097422306154,
   'freqs': (2, (12, 182))}),
 ('человека свободного',
  {'Dice': 0.020618556701030927,
   'MI': 2.9837097422306154,
   'freqs': (2, (182, 12))}

И, о чудо! Мера Дайса увеличилась, а в списке появились и найденные уже нами имя-фамилия главного героя, и некоторые действия, которые были в топ-списке, упорядоченном по Дайсу. Отличие состоит в том, что мера MI оценивает влияние одного слова на второе в паре - поэтому полученные в этом списке действия гораздо более точно указывают на личность Артура, чем действия в списке по Дайсу.

Таким образом, увеличение частоты встречаемости второго слова в биграмме можно сделать полученный список менее конкретным и более приближенным к списку по Дайсу.

In [32]:
[b for b in mi_list if b[1]['freqs'][0] > 5][:20]

[('хороших людей',
  {'Dice': 0.05309734513274336,
   'MI': 2.7830390467684643,
   'freqs': (6, (44, 182))}),
 ('такого человека',
  {'Dice': 0.04627249357326478,
   'MI': 1.6399749955295209,
   'freqs': (9, (207, 182))}),
 ('человека который',
  {'Dice': 0.047244094488188976,
   'MI': 1.4734784798799634,
   'freqs': (12, (182, 326))}),
 ('эти люди',
  {'Dice': 0.03286384976525822,
   'MI': 1.224211135220782,
   'freqs': (7, (244, 182))}),
 ('одних людей',
  {'Dice': 0.028846153846153848,
   'MI': 1.1119075653290242,
   'freqs': (6, (234, 182))}),
 ('человек а',
  {'Dice': 0.026168224299065422,
   'MI': 0.854911303580687,
   'freqs': (7, (182, 353))}),
 ('людьми не',
  {'Dice': 0.015414258188824663,
   'MI': 0.10264037699676444,
   'freqs': (8, (182, 856))}),
 ('людям в',
  {'Dice': 0.009216589861751152,
   'MI': -0.4768042165879593,
   'freqs': (7, (182, 1337))}),
 ('человек и',
  {'Dice': 0.008854454897620365,
   'MI': -0.5383523416253313,
   'freqs': (8, (182, 1625))})]